# Data Preprocessing Notebook

### Imports and helper functions

In [2]:
import numpy as np
import os

### Defining the data directory and parameters

In [ ]:
data_dir = r"C:\Users\smearlab\clickbait-ephys"
save_dir = r"C:\Users\smearlab\clickbait-ephys\"
mice = ['6000', '6001', '6002', '6003']

os.makedirs(save_dir, exist_ok=True)

### Extracting the OpenEphys data (sniff, ephys, TTLs) and resturcturing the data

In [8]:
open_ephys_dir = os.path.join(data_dir, "open-ephys")
files = os.listdir(open_ephys_dir)

# looping through the mice and extracting all the sessions
for mouse in mice:
        mouse_data = []
        for file in files:
            if file.endswith(mouse):
                mouse_data.append(file)
        print(mouse_data)
    



['2025-02-03_14-25-38_6000', '2025-02-04_14-04-31_6000', '2025-02-05_14-03-34_6000', '2025-02-06_16-02-02_6000', '2025-02-07_14-11-26_6000', '2025-02-08_13-31-21_6000', '2025-02-09_12-26-57_6000', '2025-02-10_14-50-57_6000']
['2025-02-03_15-10-19_6001', '2025-02-04_14-44-05_6001', '2025-02-05_16-43-31_6001', '2025-02-06_16-39-31_6001', '2025-02-07_14-48-05_6001', '2025-02-08_14-12-33_6001', '2025-02-09_13-04-45_6001', '2025-02-10_17-00-56_6001']
['2025-02-03_15-49-38_6002', '2025-02-04_15-42-51_6002', '2025-02-05_15-27-24_6002', '2025-02-06_17-38-04_6002', '2025-02-07_15-25-47_6002', '2025-02-08_14-54-02_6002', '2025-02-09_13-43-56_6002', '2025-02-10_17-04-35_6002']
['2025-02-03_16-28-19_6003', '2025-02-04_16-23-17_6003', '2025-02-05_16-06-50_6003', '2025-02-06_18-14-11_6003', '2025-02-07_16-02-28_6003', '2025-02-08_15-32-06_6003', '2025-02-09_14-20-53_6003', '2025-02-10_17-40-38_6003']
